In [602]:
import pandas as pd   
import warnings; warnings.simplefilter('ignore')
#numpy is only is used to store the matrix not for matrix operations
import numpy as np

## Revised Simplex Method

In [603]:
#Currently the code is for maximization and greater than equal to type constraints problem

objective="max"
m=2#no of equations
n=3#no of variables
a=[[1,5,6,'<='],[1,3,2,"<="]]
b=[30,18]
Coeff=[1,2,3]

In [604]:
def convert_to_maximization_type(a,b,Coeff,m,n,objective):
    if objective != 'max':
        Coeff=[-Coeff[i] for i in range(n)]
    for i in range(m):
        if a[i][-1] != '<=':
            for j in range(n):
                a[i][j]=-a[i][j]
            b[i]=-b[i]
    return a,b,Coeff,m,n
                
                
    

In [605]:


#1
def initialize_simplex_table(a, b, m, n, Coeff,objective):
    convert_dict={'x'+str(i+1):float for i in range(n)}
    jconvert_dict={'x'+str(i+1):float for i in range(n)}
    a,b,Coeff,m,n=convert_to_maximization_type(a,b,Coeff,m,n,objective)
    Cj_dict={'x'+str(i+1):[Coeff[i]] for i in range(n)}
    zj_cj_dict={'x'+str(i+1):[-Coeff[i]] for i in range(n)}
    Zj_dict={'x'+str(i+1):[0] for i in range(n)}
    for i in range(m):
        Cj_dict['s'+str(i+1)]=[0]
        zj_cj_dict['s'+str(i+1)]=[0]
        Zj_dict['s'+str(i+1)]=[0]
        convert_dict['s'+str(i+1)]=float
        jconvert_dict['s'+str(i+1)]=float
    Cj_dict['sol']=0
    zj_cj_dict['sol']=0
    Zj_dict['sol']=0   
    convert_dict['sol']=float
    jconvert_dict['sol']=float
    #Cj_dict.extend({0 for i in range(m)])
    #Zj=[0 for i in range(m+n)]
    data = {'CBi':[0 for i in range(m)]} 
    convert_dict['CBi']=float
    data['basic_var']=['s'+str(i+1) for i in range(m)]
    convert_dict['basic_var']=str
    for i in range(n):
        data['x'+str(i+1)]=[a[j][i] for j in range(m)]
    #augmenting data with slack variables
    for i in range(m):
        data['s'+str(i+1)]=[float(i==j) for j in range(m)]
    data['sol']=[b[j] for j in range(m)]
    data['ratio']=[0 for j in range(m)]
    convert_dict['ratio']=float
    df = pd.DataFrame(data, index=[str(i+1) for i in range(m)]) 
    Zj = pd.DataFrame(Zj_dict, index=['Zj']) 
    Cj = pd.DataFrame(Cj_dict, index=['Cj'])
    zj_cj = pd.DataFrame(zj_cj_dict, index=['Zj-Cj'])
    
    df = df.astype(convert_dict) 
    
    Zj = Zj.astype(jconvert_dict)
    Cj = Cj.astype(jconvert_dict)
    zj_cj = zj_cj.astype(jconvert_dict)
    B_inv=np.identity(m)
    basic_var=set(['s'+str(i+1) for i in range(m)])
    non_basic_var=set(['x'+str(i+1) for i in range(n)])
    return df,Cj,Zj,B_inv,zj_cj,basic_var,non_basic_var
  

In [606]:
#printing initial simplex table
def print_initial_table(df, Cj, Zj, zj_cj):
    print("initial simplex table:")
    frames = [df, Cj, Zj, zj_cj]
    result = pd.concat(frames)    
    print(result)

### Finding new B<sup>-1</sup> using product of inverse method

In [607]:
def matrix_mult(X_,Y_):
    #X_->(m,l)
    #Y_->(l,n)

    my,ny=Y_.shape
    mx,nx=X_.shape
    
    if nx!=my:
        raise Exception("Matrix multiplication not possible")
    l=my
    m=mx
    n=ny  
    result=np.zeros((m,n))
    for i in range(m):            
        for j in range(n):           
            for k in range(l):               
                result[i,j] += (X_[i,k] * Y_[k,j])
                
   
    return result


In [608]:
#Finding new B inverse using product of inverse method

def Er(B_inv, c, r):
    m,n=B_inv.shape
    if m!=n:
        raise Exception("B_inv is not a square matrix")
    e=matrix_mult(B_inv,c.reshape(m,1))    
    er=np.identity(m)
    #Calulating eta
    for i in range(m):
        if i!=r:
            er[i,r]=(-(e[i]/e[r]))
        else:
            er[i,r]=(1/e[r])        
    
    return er    
    

In [609]:
def new_B_inv(er, prev_B_inv):
    return matrix_mult(er, prev_B_inv)


In [610]:
def Y(Cb, B_inv):
    m,n=B_inv.shape
    if m!=n:
        raise Exception("B_inv is not a square matrix")
    return matrix_mult(Cb.reshape(1,m),B_inv)

In [611]:
def col_name(i,n):
    if (i+1)<=n:
        return 'x'+str(i+1)
    else:
        return 's'+str(i+1-n)

In [612]:
def Zj_Cj(Cj,  zj_cj,Cb,B_inv):
    l=['x'+str(i+1) for i in range(n)]
    l.extend(['s'+str(i+1) for i in range(m)])   
    l.append('sol')
    for i in l:
        Pj=np.array(list(df[i]))
        y=Y(Cb, B_inv)
        zj=matrix_mult(y.reshape(1,m),Pj.reshape(m,1))
        zj_cj[i]['Zj-Cj']=zj-Cj[i]['Cj'] 
    
    return zj_cj


In [613]:
def Xb(B_inv,b):
    m,n=B_inv.shape
    if m!=n:
        raise Exception("B_inv is not a square matrix")
    return matrix_mult(B_inv,b.reshape(m,1))

def Z(Cb,Xb):
    return matrix_mult(Cb.reshape(1,m),Xb.reshape(m,1))
    

In [614]:
def Aj(B_inv, Pj):
    m,n=B_inv.shape
    if m!=n:
        raise Exception("B_inv is not a square matrix")
    return matrix_mult(B_inv,Pj.reshape(m,1))


In [615]:
#Check is optimal solution is reached or not
def is_optimal(val):   
    if val>=0:
        print("Optimality is reached")
        return True
    else:
        print("Lets proceed to next iteration")
        return False

In [616]:
def entering_var(zj_cj, non_basic_var):
    val_min=999999
    id_min=None
    for val in non_basic_var:
        if val_min>zj_cj[val]['Zj-Cj']:
            val_min=zj_cj[val]['Zj-Cj']
            id_min=val
    return id_min,val_min
    
    

In [617]:
def leaving_var(xb,aj,m):
    val_min=999999
    id_min=None
    val_aj=None
    for k in range(m):
        if val_min>(xb[k]/aj[k])>=0:
            val_min=(xb[k]/aj[k])
            id_min=k
            val_aj=aj[k]
    return id_min,val_aj
    

In [618]:
def is_bounded(ajk):
    if ajk<=0:
        print("Problem is unbounded, we can't proceed further")
        return False
    return True
    
        

In [619]:
def revised_simplex_method(df,B_inv, Cj,zj_cj, basic_var, non_basic_var,m,objective="max"):
    #step1
    for i in range(1,10):
        print(f"----------------iteration{i}-------------------------")
        print("B inverse: ")
        print(B_inv)    
        print("List of basic variables: ",basic_var)
        print("List of non basic variables: ",non_basic_var)
        Cb=np.array(list(df['CBi']))
        print("Cb: ", Cb)
        y=Y(Cb,B_inv)
        print("Y :", y)
        zj_cj=Zj_Cj(Cj, zj_cj,Cb,B_inv)
        ev,val=entering_var(zj_cj, non_basic_var)       
        #step2
        pj=np.array(list(df[ev]))
        print("Pj:",pj)
        aj=Aj(B_inv, pj)
        print("aj:",aj)
        b=np.array(list(df['sol']))
        print("b:",b)
        xb=Xb(B_inv,b)
        print("Xb:",xb)
        z=Z(Cb,xb)
        if objective=="max":
            print("z->objective function: ",z)
        else:
            print("z->objective function: ",-z)
        r,ajk=leaving_var(xb,aj,m)
        lv=df['basic_var'][r]
        print("The entering variable is: ", ev)
        print("The leaving variable is: ", lv)
        if not is_optimal(val) and is_bounded(ajk):                
            #step 3
            er=Er(B_inv, pj, r)            
            B_inv=new_B_inv(er, B_inv)
            basic_var.remove(lv)
            basic_var.add(ev)
            non_basic_var.remove(ev)
            non_basic_var.add(lv)
            df['CBi'][r]=Cj[ev]['Cj']
            df['basic_var'][r]=ev            
        else:
            break

In [620]:
df,Cj,Zj,B_inv,zj_cj,basic_var,non_basic_var = initialize_simplex_table(a, b, m, n, Coeff,objective)
print_initial_table(df,Cj,Zj,zj_cj)
revised_simplex_method(df,B_inv, Cj,zj_cj, basic_var, non_basic_var,m)   
    

initial simplex table:
       CBi basic_var   x1   x2   x3   s1   s2   sol  ratio
1      0.0        s1  1.0  5.0  6.0  1.0  0.0  30.0    0.0
2      0.0        s2  1.0  3.0  2.0  0.0  1.0  18.0    0.0
Cj     NaN       NaN  1.0  2.0  3.0  0.0  0.0   0.0    NaN
Zj     NaN       NaN  0.0  0.0  0.0  0.0  0.0   0.0    NaN
Zj-Cj  NaN       NaN -1.0 -2.0 -3.0  0.0  0.0   0.0    NaN
----------------iteration1-------------------------
B inverse: 
[[1. 0.]
 [0. 1.]]
List of basic variables:  {'s1', 's2'}
List of non basic variables:  {'x1', 'x3', 'x2'}
Cb:  [0. 0.]
Y : [[0. 0.]]
Pj: [6. 2.]
aj: [[6.]
 [2.]]
b: [30. 18.]
Xb: [[30.]
 [18.]]
z->objective function:  [[0.]]
The entering variable is:  x3
The leaving variable is:  s1
Lets proceed to next iteration
----------------iteration2-------------------------
B inverse: 
[[ 0.16666667  0.        ]
 [-0.33333333  1.        ]]
List of basic variables:  {'x3', 's2'}
List of non basic variables:  {'x1', 's1', 'x2'}
Cb:  [3. 0.]
Y : [[0.5 0. ]]
Pj: [1.

## Menu Program

### Input: 
<ol>
    <li>First line will contain number of test cases t</li>
    <li>Format for next t is as follows</li>
    <li>
        <ol>
            <li> Objective of the LPP: min or max</li>
            <li>2 spaced integers m,n denoting number of equations and number of variables respectively</li>
            <li>For next m lines, each line contains n integers (i.e) ai_1 to ai_n followed by the type of inequality &le; or &ge;</li>
            <li>m spaced integers b1 to bm</li>
            <li>n spaced integers denoting coefficients of objective function</li>
        </ol>
    </li>
</ol>

### output:
<ol>
    <li>List of simplex tables at each iteration consisting of <br/>(a) initial table <br/> (b) List of basic and non-basic variables for ith iteration  <br/>(c) optimal solution </li>
</ol>
    
    

In [552]:
#program to take input
def input_ai(val):
    if not (val == "<=" or val == ">="):
        return float(val)
    else:
        return val

t=int(input())
for i in range(t):
    print("Enter input in above format:")
    objective=input()
    m,n = map(int, input().split())
    a=[]
    for j in range(m):
        ai=list(map(input_ai, input().split()))
        a.append(ai)
    b=list(map(float, input().split()))
    Coeff=list(map(float, input().split()))
    print("Here is the output:")
    df,Cj,Zj,B_inv,zj_cj,basic_var,non_basic_var = initialize_simplex_table(a, b, m, n, Coeff,objective)
    print_initial_table(df,Cj,Zj,zj_cj)
    revised_simplex_method(df,B_inv, Cj,zj_cj, basic_var, non_basic_var,m)

In [621]:
#a
objective="max"
m=3#no of equations
n=3#no of variables
a=[[3,5,2,'<='],[4,4,4,"<="],[2,4,5,"<="]]
b=[60,72,100]
Coeff=[5,10,8]

df,Cj,Zj,B_inv,zj_cj,basic_var,non_basic_var = initialize_simplex_table(a, b, m, n, Coeff,objective)
print_initial_table(df,Cj,Zj,zj_cj)
revised_simplex_method(df,B_inv, Cj,zj_cj, basic_var, non_basic_var,m)   
    

initial simplex table:
       CBi basic_var   x1    x2   x3   s1   s2   s3    sol  ratio
1      0.0        s1  3.0   5.0  2.0  1.0  0.0  0.0   60.0    0.0
2      0.0        s2  4.0   4.0  4.0  0.0  1.0  0.0   72.0    0.0
3      0.0        s3  2.0   4.0  5.0  0.0  0.0  1.0  100.0    0.0
Cj     NaN       NaN  5.0  10.0  8.0  0.0  0.0  0.0    0.0    NaN
Zj     NaN       NaN  0.0   0.0  0.0  0.0  0.0  0.0    0.0    NaN
Zj-Cj  NaN       NaN -5.0 -10.0 -8.0  0.0  0.0  0.0    0.0    NaN
----------------iteration1-------------------------
B inverse: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
List of basic variables:  {'s3', 's1', 's2'}
List of non basic variables:  {'x1', 'x3', 'x2'}
Cb:  [0. 0. 0.]
Y : [[0. 0. 0.]]
Pj: [5. 4. 4.]
aj: [[5.]
 [4.]
 [4.]]
b: [ 60.  72. 100.]
Xb: [[ 60.]
 [ 72.]
 [100.]]
z->objective function:  [[0.]]
The entering variable is:  x2
The leaving variable is:  s1
Lets proceed to next iteration
----------------iteration2-------------------------
B inverse: 
[[ 0.2  0.   0.

In [622]:
#b
objective="max"
m=4#no of equations
n=2#no of variables
a=[[2,1,'<='],[1,1,"<="],[1,0,"<="],[0,1,"<="]]
b=[1000,800,400,700]
Coeff=[4,3]

df,Cj,Zj,B_inv,zj_cj,basic_var,non_basic_var = initialize_simplex_table(a, b, m, n, Coeff,objective)
print_initial_table(df,Cj,Zj,zj_cj)
revised_simplex_method(df,B_inv, Cj,zj_cj, basic_var, non_basic_var,m)   
    

initial simplex table:
       CBi basic_var   x1   x2   s1   s2   s3   s4     sol  ratio
1      0.0        s1  2.0  1.0  1.0  0.0  0.0  0.0  1000.0    0.0
2      0.0        s2  1.0  1.0  0.0  1.0  0.0  0.0   800.0    0.0
3      0.0        s3  1.0  0.0  0.0  0.0  1.0  0.0   400.0    0.0
4      0.0        s4  0.0  1.0  0.0  0.0  0.0  1.0   700.0    0.0
Cj     NaN       NaN  4.0  3.0  0.0  0.0  0.0  0.0     0.0    NaN
Zj     NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0     0.0    NaN
Zj-Cj  NaN       NaN -4.0 -3.0  0.0  0.0  0.0  0.0     0.0    NaN
----------------iteration1-------------------------
B inverse: 
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
List of basic variables:  {'s3', 's4', 's1', 's2'}
List of non basic variables:  {'x1', 'x2'}
Cb:  [0. 0. 0. 0.]
Y : [[0. 0. 0. 0.]]
Pj: [2. 1. 1. 0.]
aj: [[2.]
 [1.]
 [1.]
 [0.]]
b: [1000.  800.  400.  700.]
Xb: [[1000.]
 [ 800.]
 [ 400.]
 [ 700.]]
z->objective function:  [[0.]]
The entering variable is:  x1
The leaving v

In [623]:
#c
objective="max"
m=2#no of equations
n=4#no of variables
a=[[2,2,5,1,'<='],[3,3,4,0,"<="]]
b=[12,11]
Coeff=[3,3,2,1]

df,Cj,Zj,B_inv,zj_cj,basic_var,non_basic_var = initialize_simplex_table(a, b, m, n, Coeff,objective)
print_initial_table(df,Cj,Zj,zj_cj)
revised_simplex_method(df,B_inv, Cj,zj_cj, basic_var, non_basic_var,m)   
    

initial simplex table:
       CBi basic_var   x1   x2   x3   x4   s1   s2   sol  ratio
1      0.0        s1  2.0  2.0  5.0  1.0  1.0  0.0  12.0    0.0
2      0.0        s2  3.0  3.0  4.0  0.0  0.0  1.0  11.0    0.0
Cj     NaN       NaN  3.0  3.0  2.0  1.0  0.0  0.0   0.0    NaN
Zj     NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0   0.0    NaN
Zj-Cj  NaN       NaN -3.0 -3.0 -2.0 -1.0  0.0  0.0   0.0    NaN
----------------iteration1-------------------------
B inverse: 
[[1. 0.]
 [0. 1.]]
List of basic variables:  {'s1', 's2'}
List of non basic variables:  {'x1', 'x3', 'x4', 'x2'}
Cb:  [0. 0.]
Y : [[0. 0.]]
Pj: [2. 3.]
aj: [[2.]
 [3.]]
b: [12. 11.]
Xb: [[12.]
 [11.]]
z->objective function:  [[0.]]
The entering variable is:  x1
The leaving variable is:  s2
Lets proceed to next iteration
----------------iteration2-------------------------
B inverse: 
[[ 1.         -0.66666667]
 [ 0.          0.33333333]]
List of basic variables:  {'x1', 's1'}
List of non basic variables:  {'s2', 'x3', 'x4', 

In [624]:
#d
objective="max"
m=3#no of equations
n=3#no of variables
a=[[2,3,0,'<='],[0,2,5,"<="],[3,2,4,"<="]]
b=[8,10,15]
Coeff=[3,5,4]

df,Cj,Zj,B_inv,zj_cj,basic_var,non_basic_var = initialize_simplex_table(a, b, m, n, Coeff,objective)
print_initial_table(df,Cj,Zj,zj_cj)
revised_simplex_method(df,B_inv, Cj,zj_cj, basic_var, non_basic_var,m)   
    

initial simplex table:
       CBi basic_var   x1   x2   x3   s1   s2   s3   sol  ratio
1      0.0        s1  2.0  3.0  0.0  1.0  0.0  0.0   8.0    0.0
2      0.0        s2  0.0  2.0  5.0  0.0  1.0  0.0  10.0    0.0
3      0.0        s3  3.0  2.0  4.0  0.0  0.0  1.0  15.0    0.0
Cj     NaN       NaN  3.0  5.0  4.0  0.0  0.0  0.0   0.0    NaN
Zj     NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0   0.0    NaN
Zj-Cj  NaN       NaN -3.0 -5.0 -4.0  0.0  0.0  0.0   0.0    NaN
----------------iteration1-------------------------
B inverse: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
List of basic variables:  {'s3', 's1', 's2'}
List of non basic variables:  {'x1', 'x3', 'x2'}
Cb:  [0. 0. 0.]
Y : [[0. 0. 0.]]
Pj: [3. 2. 2.]
aj: [[3.]
 [2.]
 [2.]]
b: [ 8. 10. 15.]
Xb: [[ 8.]
 [10.]
 [15.]]
z->objective function:  [[0.]]
The entering variable is:  x2
The leaving variable is:  s1
Lets proceed to next iteration
----------------iteration2-------------------------
B inverse: 
[[ 0.33333333  0.          0.      

In [625]:
#e
objective="min"
m=3#no of equations
n=4#no of variables
a=[[2,2,5,1,'>='],[3,-3,4,0,">="],[4,-2,3,-1,">="]]
b=[43,11,25]
Coeff=[3,3,2,-1]

df,Cj,Zj,B_inv,zj_cj,basic_var,non_basic_var = initialize_simplex_table(a, b, m, n, Coeff,objective)
print_initial_table(df,Cj,Zj,zj_cj)
revised_simplex_method(df,B_inv, Cj,zj_cj, basic_var, non_basic_var,m,objective="min")   
    

initial simplex table:
       CBi basic_var   x1   x2   x3   x4   s1   s2   s3   sol  ratio
1      0.0        s1 -2.0 -2.0 -5.0 -1.0  1.0  0.0  0.0 -43.0    0.0
2      0.0        s2 -3.0  3.0 -4.0  0.0  0.0  1.0  0.0 -11.0    0.0
3      0.0        s3 -4.0  2.0 -3.0  1.0  0.0  0.0  1.0 -25.0    0.0
Cj     NaN       NaN -3.0 -3.0 -2.0  1.0  0.0  0.0  0.0   0.0    NaN
Zj     NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0    NaN
Zj-Cj  NaN       NaN  3.0  3.0  2.0 -1.0  0.0  0.0  0.0   0.0    NaN
----------------iteration1-------------------------
B inverse: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
List of basic variables:  {'s3', 's1', 's2'}
List of non basic variables:  {'x1', 'x3', 'x4', 'x2'}
Cb:  [0. 0. 0.]
Y : [[0. 0. 0.]]
Pj: [-1.  0.  1.]
aj: [[-1.]
 [ 0.]
 [ 1.]]
b: [-43. -11. -25.]
Xb: [[-43.]
 [-11.]
 [-25.]]
z->objective function:  [[-0.]]
The entering variable is:  x4
The leaving variable is:  s1
Lets proceed to next iteration
Problem is unbounded, we can't proceed furthe